In [1]:
# Example of saving and restoring TensorFlow variables/operations/weights
# 2017-04-10 jkang
# Python3.5
# TensorFlow1.0.1
#
# Simple linear regression
# >> y = w*x + b
#
# ref: 
# - https://www.tensorflow.org/programmers_guide/variables
# - https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc

import tensorflow as tf
import numpy as np
import os

In [2]:
# Define input/output
x_data = np.array([1,2,3,4,5,6,7,8,9,10], dtype=np.float32)
y_data = np.array([5.1,6.9,8,11.5,13.1,14.7,15.9,20,20.8,22], dtype=np.float32)

# Define variables
x = tf.placeholder(tf.float32, shape=(), name='x_data'); print('x', x)
y = tf.placeholder(tf.float32, shape=(), name='y_data'); print('y', y)

w = tf.Variable(np.random.random(), dtype=tf.float32, name='Weight'); print('w', w)
b = tf.Variable(np.random.random(), dtype=tf.float32, name='Bias'); print('b', b)

y_pred = tf.add(tf.multiply(x, w), b)
loss = tf.square(tf.subtract(y, y_pred), name='loss')
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss); print('optimizer:\n', optimizer)

# Define Saver object
saver = tf.train.Saver()
# saver = tf.train.Saver({'v1': v1}) # saving only specified variable

# Make result folder
if not os.path.exists('result'):
    os.makedirs('result')

init = tf.global_variables_initializer()

x Tensor("x_data:0", shape=(), dtype=float32)
y Tensor("y_data:0", shape=(), dtype=float32)
w Tensor("Weight/read:0", shape=(), dtype=float32)
b Tensor("Bias/read:0", shape=(), dtype=float32)
optimizer:
 name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Weight/ApplyGradientDescent"
input: "^GradientDescent/update_Bias/ApplyGradientDescent"



In [3]:
# Run session
with tf.Session() as sess:
    sess.run(init)
    for i in range(20):
        total_loss = 0
        for x_curr, y_curr in zip(x_data, y_data):
            _, l = sess.run([optimizer, loss], feed_dict={x: x_curr, y: y_curr})
            total_loss += l
        print("Epoch {0}: {1}".format(i, total_loss))
    saver.save(sess, './result/dataAll.ckpt')

# List saved files
os.listdir('result')

Epoch 0: 180.78787843883038
Epoch 1: 19.51405463460833
Epoch 2: 19.019673123024404
Epoch 3: 18.55915463063866
Epoch 4: 18.130264250095934
Epoch 5: 17.73079610662535
Epoch 6: 17.358743955526734
Epoch 7: 17.012191938469186
Epoch 8: 16.68944007786922
Epoch 9: 16.388848135713488
Epoch 10: 16.108919837512076
Epoch 11: 15.848211880773306
Epoch 12: 15.605369564145803
Epoch 13: 15.379253651946783
Epoch 14: 15.168646726757288
Epoch 15: 14.972509441897273
Epoch 16: 14.789853818714619
Epoch 17: 14.619794154539704
Epoch 18: 14.461368504911661
Epoch 19: 14.313879705965519


['checkpoint',
 'dataAll.ckpt.data-00000-of-00001',
 'dataAll.ckpt.index',
 'dataAll.ckpt.meta']

In [4]:
# Restore saved data
new_saver = tf.train.import_meta_graph('./result/dataAll.ckpt.meta') # load meta graph
graph = tf.get_default_graph()

# Restore variables/operations
x_restore = graph.get_tensor_by_name('x_data:0'); print('x =', x_restore)
y_restore = graph.get_tensor_by_name('y_data:0'); print('y =', y_restore)
optimizer_restore = graph.get_operation_by_name('GradientDescent'); print(optimizer_restore)

# Restore weight/bias (they must be restored within tf.Session)
with tf.Session() as sess:
    sess.run(init)
    new_saver.restore(sess, './result/dataAll.ckpt')
    
    # Print all stored items
    for key in graph.get_all_collection_keys():
        for var in graph.get_collection(key):
            print(var)


x = Tensor("x_data:0", shape=(), dtype=float32)
y = Tensor("y_data:0", shape=(), dtype=float32)
name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Weight/ApplyGradientDescent"
input: "^GradientDescent/update_Bias/ApplyGradientDescent"

Tensor("Weight/read:0", shape=(), dtype=float32)
Tensor("Bias/read:0", shape=(), dtype=float32)
Tensor("Weight/read:0", shape=(), dtype=float32)
Tensor("Bias/read:0", shape=(), dtype=float32)
Tensor("Weight/read:0", shape=(), dtype=float32)
Tensor("Bias/read:0", shape=(), dtype=float32)
Tensor("Weight/read:0", shape=(), dtype=float32)
Tensor("Bias/read:0", shape=(), dtype=float32)
name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Weight/ApplyGradientDescent"
input: "^GradientDescent/update_Bias/ApplyGradientDescent"

name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Weight/ApplyGradientDescent"
input: "^GradientDescent/update_Bias/ApplyGradientDescent"

